### COMP SCI 524
##### Homework 1 
Jingde Wan

### Q1. Construction with constraints

Let $x_{i,j}$ be the number of worker-months of month i project j.

In [99]:
using JuMP

model_c = Model()

# no more than 6 workers in a project
@variable(model_c, 0 <= x11 <= 6)
@variable(model_c, 0 <= x12 <= 6)
@variable(model_c, 0 <= x13 <= 6)
@variable(model_c, 0 <= x21 <= 6)
@variable(model_c, 0 <= x22 <= 6)
@variable(model_c, 0 <= x23 <= 6)
@variable(model_c, 0 <= x31 <= 6)
@variable(model_c, 0 <= x32 <= 6)
@variable(model_c, 0 <= x33 <= 6)
@variable(model_c, 0 <= x41 <= 6)
@variable(model_c, 0 <= x42 <= 6)
@variable(model_c, 0 <= x43 <= 6)

# time requirement for each project
@constraint(model_c, x41 == 0)
@constraint(model_c, x33 == 0)
@constraint(model_c, x43 == 0)

#labor requirement for each project
@constraint(model_c, x11+x21+x31+x41 >= 8)
@constraint(model_c, x12+x22+x32+x42 >= 10)
@constraint(model_c, x13+x23+x33+x43 >= 12)

# 8 workers are available for each month
@constraint(model_c, x11+x12+x13 <= 8)
@constraint(model_c, x21+x22+x23 <= 8)
@constraint(model_c, x31+x32+x33 <= 8)
@constraint(model_c, x41+x42+x43 <= 8)

@objective(model_c, Max, x11+x12+x13+x21+x22+x23+x31+x32+x33+x41+x42+x43)
model_c

A JuMP Model
Maximization problem with:
Variables: 12
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 12 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 12 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x11, x12, x13, x21, x22, x23, x31, x32, x33, x41, x42, x43

In [101]:
using Clp
set_optimizer(model_c, Clp.Optimizer)
optimize!(model_c)
println(termination_status(model_c))

OPTIMAL
Coin0506I Presolve 5 (-5) rows, 6 (-6) columns and 12 (-15) elements
Clp0006I 0  Obj 21.9 Primal inf 8.0999981 (2) Dual inf 5.999994 (6)
Clp0006I 7  Obj 30
Clp0000I Optimal - objective value 30
Coin0511I After Postsolve, objective 30, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 30 - 7 iterations time 0.002, Presolve 0.00


Therefore, all three projects can be completed on time.

### Q2. Stigler’s diet

(a) Let $x_{i}$ to denote the unit I spend on i-th food.
The minimization problem can be described as following:

In [3]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("NamedArrays")

   Updating registry at `C:\Users\jingd\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\jingd\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\jingd\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\jingd\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\jingd\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\jingd\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\jingd\.julia\environments\v1.5\Manifest.toml`


In [4]:
# STARTER CODE FOR STIGLER'S DIET PROBLEM

using NamedArrays
using CSV
using DataFrames

# import Stigler's data set
raw = CSV.read(joinpath(@__DIR__, "stigler.csv"), DataFrame)
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 2:m          # rows containing food names

nutrients = names(raw)[n_nutrients]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods, 1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lower = Dict(zip(nutrients,raw[1, n_nutrients]))
# println(lower)

# data[f,i] is the amount of nutrient i contained in food f.
# # Because we cannot construct namedarray directly from data frame
# # we need to convert it into a matrix first
data = convert(Matrix{Float64}, raw[n_foods, n_nutrients])
data = NamedArray(data, (foods, nutrients), ("foods", "nutrients"))

77×9 Named Array{Float64,2}
      foods ╲ nutrients │     Calories (1000)  …   Ascorbic Acid (mg)
────────────────────────┼────────────────────────────────────────────
Wheat Flour (Enriched)  │                44.7  …                  0.0
Macaroni                │                11.6                     0.0
Wheat Cereal (Enriched) │                11.8                     0.0
Corn Flakes             │                11.4                     0.0
Corn Meal               │                36.0                     0.0
Hominy Grits            │                28.6                     0.0
Rice                    │                21.2                     0.0
Rolled Oats             │                25.3                     0.0
White Bread (Enriched)  │                15.0                     0.0
Whole Wheat Bread       │                12.2                     0.0
Rye Bread               │                12.4                     0.0
Pound Cake              │                 8.0                 

In [20]:
using JuMP

model = Model()

@variable(model, x[1:length(n_foods)] >= 0)

@constraint(model, sum(data[i, "Calories (1000)"] * x[i] for i in 1:length(n_foods)) >= lower["Calories (1000)"])
@constraint(model, sum(data[i, "Ascorbic Acid (mg)"] * x[i] for i in 1:length(n_foods)) >= lower["Ascorbic Acid (mg)"])
@constraint(model, sum(data[i, "Calcium (g)"] * x[i] for i in 1:length(n_foods)) >= lower["Calcium (g)"])
@constraint(model, sum(data[i, "Niacin (mg)"] * x[i] for i in 1:length(n_foods)) >= lower["Niacin (mg)"])
@constraint(model, sum(data[i, "Thiamine (mg)"] * x[i] for i in 1:length(n_foods)) >= lower["Thiamine (mg)"])
@constraint(model, sum(data[i, "Iron (mg)"] * x[i] for i in 1:length(n_foods)) >= lower["Iron (mg)"])
@constraint(model, sum(data[i, "Vitamin A (1000 IU)"] * x[i] for i in 1:length(n_foods)) >= lower["Vitamin A (1000 IU)"])
@constraint(model, sum(data[i, "Riboflavin (mg)"] * x[i] for i in 1:length(n_foods)) >= lower["Riboflavin (mg)"])
@constraint(model, sum(data[i, "Protein (g)"] * x[i] for i in 1:length(n_foods)) >= lower["Protein (g)"])

@objective(model, Min, sum(x[i] for i in 1:length(n_foods)))
model










A JuMP Model
Minimization problem with:
Variables: 77
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 77 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

In [34]:
using Clp
set_optimizer(model, Clp.Optimizer)
@time optimize!(model)

println("The total cost of cheapest diet: ", objective_value(model)*365)

for i in 1:length(n_foods)
    println(foods[i], ": ", getvalue(x[i]))
end



  0.000938 seconds (1.57 k allocations: 218.656 KiB)
The total cost of cheapest diet: 39.66173154546625
Wheat Flour (Enriched): 0.02951906167648827
Macaroni: 0.0
Wheat Cereal (Enriched): 0.0
Corn Flakes: 0.0
Corn Meal: 0.0
Hominy Grits: 0.0
Rice: 0.0
Rolled Oats: 0.0
White Bread (Enriched): 0.0
Whole Wheat Bread: 0.0
Rye Bread: 0.0
Pound Cake: 0.0
Soda Crackers: 0.0
Milk: 0.0
Evaporated Milk (can): 0.0
Butter: 0.0
Oleomargarine: 0.0
Eggs: 0.0
Cheese (Cheddar): 0.0
Cream: 0.0
Peanut Butter: 0.0
Mayonnaise: 0.0
Crisco: 0.0
Lard: 0.0
Sirloin Steak: 0.0
Round Steak: 0.0
Rib Roast: 0.0
Chuck Roast: 0.0
Plate: 0.0
Liver (Beef): 0.0018925572907052643
Leg of Lamb: 0.0
Lamb Chops (Rib): 0.0
Pork Chops: 0.0
Pork Loin Roast: 0.0
Bacon: 0.0
Ham, smoked: 0.0
Salt Pork: 0.0
Roasting Chicken: 0.0
Veal Cutlets: 0.0
Salmon, Pink (can): 0.0
Apples: 0.0
Bananas: 0.0
Lemons: 0.0
Oranges: 0.0
Green Beans: 0.0
Cabbage: 0.011214435246144865
Carrots: 0.0
Celery: 0.0
Lettuce: 0.0
Onions: 0.0
Potatoes: 0.0
Spin

The total cost of my cheapest diet is $39.66 which is smaller than Stigler's.
Food making up my optimal diet includes: 
1. Wheat Flour (Enriched): 0.02951906167648827
2. Liver (Beef): 0.0018925572907052643
3. Cabbage: 0.011214435246144865
4. Spinach: 0.005007660466725203
5. Navy Beans, Dried: 0.061028563526693246

(b)

In [75]:
lower_bounds = raw[1, n_nutrients]

for j in 1:9
    println("The lower bound for ", nutrients[j], ": ", lower_bounds[j], ". The corresponding amount in optimal diet: ", sum(data[a, j] * getvalue(x[a]) for a in 1:length(n_foods)))
end



The lower bound for Calories (1000): 3.0. The corresponding amount in optimal diet: 3.0
The lower bound for Protein (g): 70. The corresponding amount in optimal diet: 147.41353494220908
The lower bound for Calcium (g): 0.8. The corresponding amount in optimal diet: 0.8000000000000002
The lower bound for Iron (mg): 12. The corresponding amount in optimal diet: 60.4669221017366
The lower bound for Vitamin A (1000 IU): 5.0. The corresponding amount in optimal diet: 5.0
The lower bound for Thiamine (mg): 1.8. The corresponding amount in optimal diet: 4.120438804838622
The lower bound for Riboflavin (mg): 2.7. The corresponding amount in optimal diet: 2.7
The lower bound for Niacin (mg): 18. The corresponding amount in optimal diet: 27.315980700288325
The lower bound for Ascorbic Acid (mg): 75. The corresponding amount in optimal diet: 74.99999999999999


The bounds which are active are:
1. Calories (1000)
2. Calcium (g)
3. Vitamin A (1000 IU)
4. Riboflavin (mg)
5. Ascorbic Acid (mg)

(c)

In [85]:
# find index of liver (beef)
for i in 1:length(n_foods)
    if foods[i] == "Liver (Beef)"
        println(i)
    end
end

# find index of milk
for j in 1:length(n_foods)
    if foods[j] == "Milk"
        println(j)
    end
end


30
14
6.1


In [94]:
using JuMP

model1 = Model()

@variable(model1, y[1:length(n_foods)] >= 0)


@constraint(model1, y[30] == 0)
@constraint(model1, y[14] >= 0.01)
@constraint(model1, sum(data[i, "Calories (1000)"] * y[i] for i in 1:length(n_foods)) >= lower["Calories (1000)"])
@constraint(model1, sum(data[i, "Ascorbic Acid (mg)"] * y[i] for i in 1:length(n_foods)) >= lower["Ascorbic Acid (mg)"])
@constraint(model1, sum(data[i, "Calcium (g)"] * y[i] for i in 1:length(n_foods)) >= lower["Calcium (g)"])
@constraint(model1, sum(data[i, "Niacin (mg)"] * y[i] for i in 1:length(n_foods)) >= lower["Niacin (mg)"])
@constraint(model1, sum(data[i, "Thiamine (mg)"] * y[i] for i in 1:length(n_foods)) >= lower["Thiamine (mg)"])
@constraint(model1, sum(data[i, "Iron (mg)"] * y[i] for i in 1:length(n_foods)) >= lower["Iron (mg)"])
@constraint(model1, sum(data[i, "Vitamin A (1000 IU)"] * y[i] for i in 1:length(n_foods)) >= lower["Vitamin A (1000 IU)"])
@constraint(model1, sum(data[i, "Riboflavin (mg)"] * y[i] for i in 1:length(n_foods)) >= lower["Riboflavin (mg)"])
@constraint(model1, sum(data[i, "Protein (g)"] * y[i] for i in 1:length(n_foods)) >= lower["Protein (g)"])

@objective(model1, Min, sum(y[i] for i in 1:length(n_foods)))
model

A JuMP Model
Minimization problem with:
Variables: 77
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 77 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Clp
Names registered in the model: x

In [95]:
using Clp
set_optimizer(model1, Clp.Optimizer)
@time optimize!(model1)

println("The total cost of cheapest diet: ", objective_value(model1)*365)

for i in 1:length(n_foods)
    println(foods[i], ": ", getvalue(y[i]))
end

  0.467654 seconds (1.33 M allocations: 67.300 MiB)
The total cost of cheapest diet: 40.86157374283626
Wheat Flour (Enriched): 0.03569701410638524
Macaroni: 0.0
Wheat Cereal (Enriched): 0.0
Corn Flakes: 0.0
Corn Meal: 0.0
Hominy Grits: 0.0
Rice: 0.0
Rolled Oats: 0.0
White Bread (Enriched): 0.0
Whole Wheat Bread: 0.0
Rye Bread: 0.0
Pound Cake: 0.0
Soda Crackers: 0.0
Milk: 0.01
Evaporated Milk (can): 0.0021523432962831466
Butter: 0.0
Oleomargarine: 0.0
Eggs: 0.0
Cheese (Cheddar): 0.0
Cream: 0.0
Peanut Butter: 0.0
Mayonnaise: 0.0
Crisco: 0.0
Lard: 0.0
Sirloin Steak: 0.0
Round Steak: 0.0
Rib Roast: 0.0
Chuck Roast: 0.0
Plate: 0.0
Liver (Beef): 0.0
Leg of Lamb: 0.0
Lamb Chops (Rib): 0.0
Pork Chops: 0.0
Pork Loin Roast: 0.0
Bacon: 0.0
Ham, smoked: 0.0
Salt Pork: 0.0
Roasting Chicken: 0.0
Veal Cutlets: 0.0
Salmon, Pink (can): 0.0
Apples: 0.0
Bananas: 0.0
Lemons: 0.0
Oranges: 0.0
Green Beans: 0.0
Cabbage: 0.01099109193703656
Carrots: 0.0
Celery: 0.0
Lettuce: 0.0
Onions: 0.0
Potatoes: 0.0
Spina

The annual total cost of my cheapest diet is $40.86.
Food making up my optimal diet includes: 
1. Wheat Flour (Enriched): 0.02951906167648827
2. Milk: 0.01
3. Evaporated Milk (can): 0.0021523432962831466
4. Cabbage: 0.01099109193703656
5. Spinach: 0.005114223881043092
6. Navy Beans, Dried: 0.04799484388291296

### 3.

(a)
let x1 be the x-coordinate of the center
let x2 be the y-coordinate of the center
let r be the radius of the museum

Equations of blue lines:

$$2x_{1} + 3x_{2} <= 2100$$
$$3x_{1} - x_{2} <= 1500$$
$$x_{1} >= 0$$
$$x_{2} <= 500$$
$$x_{2} >= 0$$
$$x_{1} <= 600$$

By Chebyshev center formula, the mathematical model is:

$\max_{x,r} ~~r $

subject to

$ a_ix + ||a_i||r\leq b_i\quad$ $\forall i$

where $$A = 
\begin{bmatrix}
2 & 3\\
3 & -1\\
0 & -1\\
0 & 1\\
-1 & 0
\end{bmatrix}$$

$$b = 
\begin{bmatrix}
2100\\
1500\\
0\\
500\\
0
\end{bmatrix}$$

(b)

In [2]:
A = [2 3; 3 -1; -1 0; 0 1; 0 -1; 1 0] 
b = [2100; 1500; 0; 500; 0; 600]

using JuMP, Clp, LinearAlgebra

m = Model(Clp.Optimizer)
@variable(m, r >= 0)           
@variable(m, x[1:2] >= 0)          
for i = 1:size(A, 1)
    @constraint(m, A[i, :]'*x + (r + 75)*norm(A[i, :]) <= b[i])
end
@objective(m, Max, r)     

optimize!(m)
center = value.(x)
radius = value(r)

println("The center of museum is:", center)
println("The radius is: ", radius)

The center of museum is:[244.0285267938019, 244.02852679380186]
The radius is: 169.02852679380186
Coin0506I Presolve 6 (0) rows, 3 (0) columns and 14 (0) elements
Clp0006I 0  Obj 0 Primal inf 150 (2) Dual inf 0.9999999 (1)
Clp0006I 4  Obj 169.02853
Clp0000I Optimal - objective value 169.02853
Clp0032I Optimal objective 169.0285268 - 4 iterations time 0.012


Plot the figure:

In [16]:
using Pkg
Pkg.add("PyPlot")

  Resolving package versions...
No Changes to `C:\Users\jingd\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\jingd\.julia\environments\v1.5\Manifest.toml`


In [3]:
using PyPlot
# Defining a function to plot the outline of the museum area
function plot_site(mode)
    plot([0,0], [0,500], "b-")
    plot([0,500], [0,0], "b-")
    plot([500,600], [0,300],"b-")
    plot([300,600], [500,300],"b-")
    plot([0,300], [500,500],"b-")
    axis("image")
    axis([-.5,6.5,-.5,5.5]*100)
    grid()
end

figure(figsize=(6,5))

plot_site(1)

plt.gcf().gca().add_artist(plt.Circle((center[1], center[2]), radius, color = "red", fill = true))

plt.gcf().gca().add_artist(plt.Circle((center[1], center[2]), radius + 75, fill = false))

┌ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1278
ERROR: LoadError: LoadError: PyCall not properly installed. Please run Pkg.build("PyCall")
Stacktrace:
 [1] error(::String) at .\error.jl:33
 [2] top-level scope at C:\Users\jingd\.julia\packages\PyCall\tqyST\src\startup.jl:44
 [3] include(::Function, ::Module, ::String) at .\Base.jl:380
 [4] include at .\Base.jl:368 [inlined]
 [5] include(::String) at C:\Users\jingd\.julia\packages\PyCall\tqyST\src\PyCall.jl:1
 [6] top-level scope at C:\Users\jingd\.julia\packages\PyCall\tqyST\src\PyCall.jl:38
 [7] include(::Function, ::Module, ::String) at .\Base.jl:380
 [8] include(::Module, ::String) at .\Base.jl:368
 [9] top-level scope at none:2
 [10] eval at .\boot.jl:331 [inlined]
 [11] eval(::Expr) at .\client.jl:467
 [12] top-level scope at .\none:3
in expression starting at C:\Users\jingd\.julia\packages\PyCall\tqyST\src\startup.jl:41
in expression starting at C:\Users\jingd\.julia\packages\PyCall\tqy

LoadError: Failed to precompile PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee] to C:\Users\jingd\.julia\compiled\v1.5\PyPlot\oatAj_QuyST.ji.